In [1]:
import pandas as pd
from auxiliary.filter_functions import filter_chandra, filter_archival, filter_gaia, filter_ned, filter_simbad, filter_erosita, filter_vizier
from astroquery.vizier import Vizier
from astroquery.simbad import Simbad, SimbadClass

In [2]:
catalogs = Vizier.find_catalogs([
    '1RXS'
])

print({
    k: v.description for k, v in catalogs.items()
})

{'IX/10A': 'ROSAT All-Sky Bright Source Catalogue (1RXS) (Voges+ 1999)', 'IX/29': 'ROSAT All-Sky Survey Faint Source Catalog (Voges+ 2000)'}


In [3]:
detections = pd.read_csv('output/detections_w20.txt', header=0, sep=' ', dtype=str)

detection = detections[detections['ObsId'] == '20925'].iloc[0]

filter_archival(detection, True) 

Empty TableList


False

In [4]:
Simbad.list_votable_fields()

--NOTES--

1. The parameter filtername must correspond to an existing filter. Filters include: B,V,R,I,J,K.  They are checked by SIMBAD but not astroquery.simbad

2. Fields beginning with rvz display the data as it is in the database. Fields beginning with rv force the display as a radial velocity. Fields beginning with z force the display as a redshift

3. For each measurement catalog, the VOTable contains all fields of the first measurement. When applicable, the first measurement is the mean one. 

Available VOTABLE fields:

bibcodelist(y1-y2)
biblio
cel
cl.g
coo(opt)
coo_bibcode
coo_err_angle
coo_err_maja
coo_err_mina
coo_qual
coo_wavelength
coordinates
dec(opt)
dec_prec
diameter
dim
dim_angle
dim_bibcode
dim_incl
dim_majaxis
dim_minaxis
dim_qual
dim_wavelength
dimensions
distance
distance_result
einstein
fe_h
flux(filtername)
flux_bibcode(filtername)
flux_error(filtername)
flux_name(filtername)
flux_qual(filtername)
flux_system(filtername)
flux_unit(filtername)
fluxdata(filtername)

In [5]:
detections = pd.read_csv('output/detections_w20.txt',
                         header=0, sep=' ', dtype=str)

detection = detections[detections['ObsId'] == '8490'].iloc[1]

filter_vizier(detection, True)

table 0
  USNO-A2.0    RAJ2000    DEJ2000   ACTflag Mflag Bmag Rmag  Epoch  
                 deg        deg                   mag  mag     yr   
------------- ---------- ---------- ------- ----- ---- ---- --------
0450-15666527 201.469598 -43.096200               18.9 16.7 1979.682
table 1
   GSC2.2     RA_ICRS_   DE_ICRS_   Epoch    Rmag e_Rmag Bjmag e_Bjmag Class   a      e  
                deg        deg        yr     mag   mag    mag    mag          pix        
------------ ---------- ---------- -------- ----- ------ ----- ------- ----- ------ -----
S23233014232 201.469586 -43.096252 1991.225 17.09   0.39 17.92    0.39     0   2.86  0.12
table 2
 USNO-B1.0    RAJ2000    DEJ2000   e_RAJ2000 e_DEJ2000 Epoch    pmRA     pmDE   Ndet B1mag R1mag B2mag R2mag  Imag
                deg        deg        mas       mas      yr   mas / yr mas / yr       mag   mag   mag   mag   mag 
------------ ---------- ---------- --------- --------- ------ -------- -------- ---- ----- ----- ----- ----- -

True